In [ ]:
import pandas as pd

In [ ]:
sentiment = pd.read_csv(r"C:\Users\matti\OneDrive\Thesis\Data\FINAL_daily_sentiment_GOOG.csv")
sentiment["date"] = pd.to_datetime(sentiment["date"])

price_indicators = pd.read_csv(r"C:\Users\matti\OneDrive\Thesis\Data\FINAL_price_indicators.csv")
price_indicators["date"] =  pd.to_datetime(price_indicators["date"])

print(f"Sentiment Data: /n{sentiment.columns}")
print()
print(f"Price and Indicator Data: /n{price_indicators.columns}")

In [ ]:
#Sentiment Data

earliest_date_s = sentiment["date"][0]
latest_date_s = sentiment["date"][len(sentiment) - 1]

print(f"Date for Sentiment: {earliest_date_s} - {latest_date_s}, for a total of: {len(sentiment)} datapoints")

In [ ]:
#Price and Indicators Data 

earliest_date_p = price_indicators.loc[price_indicators["date"] == "2020-01-02", "date"].iloc[0]
latest_date_p = price_indicators["date"].iloc[-1]

filtered = price_indicators[(price_indicators["date"] >= earliest_date_p) & (price_indicators["date"] <= latest_date_p)]

print(f"Date for Sentiment: {earliest_date_p} - {latest_date_p}, for a total of {len(filtered)} datapoints")

In [ ]:
#Filter price and indicators to same range as sentiment
final_filtered_pi = filtered[(filtered["date"] >= earliest_date_s) & (filtered["date"] <= latest_date_s)]

#check that length is the same as sentiment
print(f"Price and Indicator range: {final_filtered_pi['date'].min()} - {final_filtered_pi['date'].max()}, for a total of {len(final_filtered_pi)}")
print(f"Sentiment range: {earliest_date_s} - {latest_date_s}, for a total of {len(sentiment)}")

In [ ]:
"""
Sentiment data also contains data for holidays, while price and indicator only has trading days which exclude holidays
"""

#So let"s remove the holidays from the sentiment data
#We can check this with difference:

# Use pandas Index to access `.difference`
dates_not_in_price = pd.Index(sentiment["date"]).difference(pd.Index(final_filtered_pi["date"]))

print(dates_not_in_price)
#These are all the dates present at the end of IndicatorData.ipynb which all correspond to a USA nationla holiday where Trading is closed

#Remove these rows 
sentiment = sentiment[~sentiment["date"].isin(dates_not_in_price)]

dates_not_in_price2 = pd.Index(sentiment["date"]).difference(pd.Index(final_filtered_pi["date"]))

print(dates_not_in_price2)
print(sentiment.columns)

In [ ]:
#Double check date ranges now after removing public holidays
print(f"Price and Indicator range: {final_filtered_pi['date'].min()} - {final_filtered_pi['date'].max()}, for a total of {len(final_filtered_pi)}")
print(f"Sentiment range: {earliest_date_s} - {latest_date_s}, for a total of {len(sentiment)}")

all_dates_present = sentiment["date"].isin(final_filtered_pi["date"]).all()
print(all_dates_present)

all_dates_present2 = final_filtered_pi["date"].isin(sentiment["date"]).all()
print(all_dates_present2)

In [ ]:
#final sort to make sure and then merge and save

sentiment = sentiment.sort_values(by="date").reset_index(drop=True)
final_filtered_pi = final_filtered_pi.sort_values(by="date").reset_index(drop=True)

merged_df = pd.merge(sentiment, final_filtered_pi, on="date", how="inner")
print(f"Merged DataFrame range: {merged_df['date'].min()} - {merged_df['date'].max()}, for a total of {len(merged_df)}")

nan_rows = merged_df[merged_df.isna().any(axis=1)]
print(nan_rows) #The first 6 rows have NaN values since sentiment is calculate on rolling window of 7 days.

#Delete these rows
merged_df.dropna(inplace=True)
print(merged_df.head())

merged_df.drop(["Unnamed: 0"], axis=1, inplace=True)

In [ ]:
merged_df.to_csv(r"C:\Users\matti\OneDrive\Thesis\Data\AGGREGATED.csv", index=True) #save to csv